In [2]:
import mlrose_hiive as mlrh
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay

random_state = 2020

In [3]:
X_cred, y_cred = fetch_openml("credit-g", return_X_y=True)

# y_cred[0], cred_mapping = pd.factorize(y_cred[0])
le = LabelEncoder()
y_cred = le.fit_transform(y_cred)

# switch labels around
# good risk: 0
# bad risk: 1
y_cred[y_cred==1] = 3  # Temp
y_cred[y_cred==0] = 1
y_cred[y_cred==3] = 0

# Create holdout test set
X_cred_train, X_cred_test, y_cred_train, y_cred_test = train_test_split(X_cred, y_cred, test_size=0.2,
                                                                        random_state=random_state,
                                                                        stratify=y_cred)
def get_data(data):
    return data["X"], data["y"], data["name"]

# Standardize
scaler_cred = StandardScaler()
scaler_cred.fit(X_cred_train)
X_cred_train = scaler_cred.transform(X_cred_train)
X_cred_test = scaler_cred.transform(X_cred_test)

# Create data sets
cred = {"X": X_cred_train, "y": y_cred_train, "name": "Credit Train"}
cred_test = {"X": X_cred_test, "y": y_cred_test, "name": "Credit Test"}

# Randomized Hill Climbing Grid Search

In [9]:
grid_search = {
    "max_iters": [1000, 2500, 5000, 10000, 25000],
    "learning_rate_init": [0.001, 0.001, 0.1, 0.5, 1],
    "hidden_layers_sizes": [[6, 6]],
    "activation": [mlrh.neural.activation.relu],
    "is_classifier": [True],
    "restarts": [0, 5, 20]
}
X, y, _ = get_data(cred)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=random_state)

runner = mlrh.NNGSRunner(x_train=X_train, y_train=y_train,
                           x_test=X_test, y_test=y_test,
                           experiment_name="full_grid_search",
                           output_directory="nn_randomized_hill_climbing/",
                           algorithm=mlrh.algorithms.random_hill_climb,
                           grid_search_parameters=grid_search,
                           iteration_list = [1, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000],
                           hidden_layer_sizes=[[6,6]],
                           bias=True,
                           early_stopping=True,
                           clip_max=1,
                           max_attempts=100,
                           generate_curves=True,
                           seed=random_state,
                           n_jobs=-1
                          )
run_stats, curves, cv_results, best_est = runner.run()

Running nngs_rhc
Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed: 192.8min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed: 749.4min finished


*****************
*** Run START ***
*****************
max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

...

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

max_iters:[5000], hidden_layers_sizes:[[6, 6]], is_classifier:[True], restarts:[20], init_state:[[ 0.97255366  0.74678389  0.01949105 -0.45632857 -0.32616254 -0.56609147
 -0.44704571 -0.31336881  0.72431787 -0.68660066 -0.71822551  0.51416056
  0.47264984 -0.28867382 -0.31781397  0.3336061  -0.56579872  0.12285396
 -0.75164244 -0.36052703  0.90642775 -0.72528642  0.13882619  0.95133097
  0.00673412  0.33532841 -0.931617   -0.08776127 -0.68829727 -0.04790207
 -0.66059512  0.79251667 -0.25321248 -0.24061414  0.71663318  0.29212211
  0.1669234   0.33670005 -0.64441476  0.69849603 -0.11525484  0.66293542
  0.52784143  0.83938195 -0.85885315 -0.68766917  0.27378846  0.11139137
 -0.61614257 -0.14868803  0.02684042 -0.46124483  0.19804004 -0.55965211
 -0.39827501 -0.90321103  0.12864012  0.87206441  0.60605528  0.3946103
 -0.07723145  0.32486922  0.49933128  0.06290978 -0.03437975 -0.94145153
  0.78856621  0.66306417  0.47568692  0.7969946  -0.53853172 -0.64007898
  0.22831624 -0.96666104 -0.

Saving: [nn_randomized_hill_climbing/full_grid_search/nngs_rhc__full_grid_search__curves_df__36C368E09CC927058D07B614AF0CD015.csv]
Saving: [nn_randomized_hill_climbing/full_grid_search/nngs_rhc__full_grid_search__cv_results_df__36C368E09CC927058D07B614AF0CD015.csv]
Saving: [nn_randomized_hill_climbing/full_grid_search/nngs_rhc__full_grid_search__grid_search_results__36C368E09CC927058D07B614AF0CD015.p]


In [10]:
best_est.best_params_

{'activation': <function mlrose_hiive.neural.activation.relu.relu(x, deriv=False)>,
 'hidden_layer_sizes': [6, 6],
 'hidden_layers_sizes': [6, 6],
 'is_classifier': True,
 'learning_rate_init': 0.5,
 'max_iters': 5000,
 'restarts': 20}